<a href="https://colab.research.google.com/github/catundchat/Summary4dialogue/blob/main/jieba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install jieba
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import jieba
import jieba.analyse
import re
import networkx as nx
import numpy as np

def generate_summary_with_textrank(conversation, topK=3):
    # 移除对话中的"A: " 和 "B: "等类似用户、咨询师的标识
    conversation_cleaned = re.sub(r"[A|B]:\s", "", conversation)
    sentences = [s.strip() for s in re.split(r'[。？！]', conversation_cleaned) if s.strip()]
    
    # 使用jieba进行分词
    words_list = [list(jieba.cut(s)) for s in sentences]

    # 构建相似度矩阵
    sim_matrix = np.zeros((len(sentences), len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_matrix[i][j] = compute_similarity(words_list[i], words_list[j])

    # 使用TextRank算法对句子进行排序
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    # 提取摘要
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary = '。'.join([sent for _, sent in ranked_sentences[:topK]])
    return summary + '。'

def compute_similarity(words1, words2):
    words1_set = set(words1)
    words2_set = set(words2)
    return len(words1_set & words2_set) / (len(words1_set | words2_set) + 1e-6)

# 示例对话
conversation = '''
情绪问题

好的，你最近有什么情绪问题吗？比如说，你是否经常感到沮丧、焦虑或者压力很大？

控制不住自己情绪怎么办

我理解你的感受。控制不住自己的情绪可能会让人感到无助和困惑。我想问一下，你最近有遇到什么事情让你感到情绪失控吗？

失恋

失恋是一件很痛苦的事情，我能理解你现在的情绪。你可以告诉我更多关于你失恋的情况吗？比如说，你们分手的原因是什么？你现在的情绪是什么样的？
'''
print()
summary = generate_summary_with_textrank(conversation)
print(summary)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.963 seconds.
DEBUG:jieba:Loading model cost 0.963 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


情绪问题

好的，你最近有什么情绪问题吗。失恋

失恋是一件很痛苦的事情，我能理解你现在的情绪。你现在的情绪是什么样的。
